📋 Problem Statement:
In today’s dynamic entertainment industry, real-time movie performance tracking is crucial for production houses, distributors, and cinema owners to make data-driven decisions. However, manually collecting and analyzing data from multiple platforms like Box Office Mojo (for box office performance) and Rotten Tomatoes (for audience & critic reviews) is time-consuming, inconsistent, and prone to errors.
The lack of an integrated system to fetch, clean, and analyze movie data hinders timely insights into box office trends, audience ratings, and movie popularity.

📊 Sample Workflow Diagram:
🔺 Box Office Mojo
(https://www.boxofficemojo.com/weekend/)
⬇
🔺 Rotten Tomatoes
(https://www.rottentomatoes.com/browse/movies_in_theaters)
⬇
[ Extract Movie Titles ]
⬇
🔺 OMDb API
(http://www.omdbapi.com/)
⬇
[ Transform & Clean Data ]
⬇
📋 CSV File | 📄 PDF Report

Develop an automated ETL (Extract, Transform, Load) pipeline that:
1. Extracts the latest movie titles from:
o Box Office Mojo (for top box office performers): https://www.boxofficemojo.com/weekend/
o Rotten Tomatoes (for critically acclaimed movies): https://www.rottentomatoes.com/browse/movies_in_theaters

In [53]:
pip install IMDbPY

Note: you may need to restart the kernel to use updated packages.


In [54]:
# ---------------------- #
# 1️⃣ Web Scraping Phase  #
# ---------------------- #


import requests
from bs4 import BeautifulSoup
def fetch_movies_from_box_office_mojo():
    print("💰 Fetching Latest Box Office Movies...")

    url = "https://www.boxofficemojo.com/weekend/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, "html.parser")

        # ✅ Target the specific class that contains movie titles
        titles = []
        movie_tags = soup.select('td.a-text-left.mojo-field-type-release a')  # Updated selector

        for movie in movie_tags:
            title = movie.text.strip()
            if title and not any(char.isdigit() for char in title):  # ✅ Exclude date-like entries
                titles.append(title)

        print(f"✅ Fetched {len(titles)} movies from Box Office Mojo!\n")
        return titles[:30]  # Limit to top 30 movies

    except requests.exceptions.RequestException as e:
        print(f"❌ Error while scraping Box Office Mojo: {e}")
        return []

# Test the function
Movies = fetch_movies_from_box_office_mojo()
print(Movies)

💰 Fetching Latest Box Office Movies...
✅ Fetched 11 movies from Box Office Mojo!

['Captain America: Brave New World', 'Captain America: Brave New World', 'Captain America: Brave New World', 'Captain America: Brave New World', 'Dog Man', 'Dog Man', 'Flight Risk', 'Mufasa: The Lion King', 'Mufasa: The Lion King', 'Den of Thieves: Pantera', 'Mufasa: The Lion King']


In [55]:


#Rotten Tomatoes (for critically acclaimed movies): https://www.rottentomatoes.com/browse/movies_in_theaters


def fetch_movies_from_rotten_tomatoes():
    print("🍅 Fetching Latest Movies from Rotten Tomatoes...")
    url = "https://www.rottentomatoes.com/browse/movies_in_theaters"

    headers = {
        "User-Agent": "Mozilla/5.0"  # Mimic a real browser
    }

    try:
        response = requests.get(url, headers=headers)
        #response.raise_for_status()  
        soup = BeautifulSoup(response.content, "html.parser")
        #print(soup)
        titles = []
        for i in soup.select('a.js-tile-link'):
            title = i.get("href").split("/")[-1].replace("-", " ").title()
            titles.append(title)

        print(f"✅ Fetched {len(titles)} movies from Rotten Tomatoes!\n")
        return titles[:30]  # Limit to 30 latest movies

    except requests.exceptions.RequestException as e:
        print(f"❌ Error while scraping Rotten Tomatoes: {e}")
        return []

# Test the function
titles = fetch_movies_from_rotten_tomatoes()
print(titles)


🍅 Fetching Latest Movies from Rotten Tomatoes...
✅ Fetched 11 movies from Rotten Tomatoes!

['Purple_Rain', 'The_Visitor_2024', 'Guns_Of_Redemption', 'Were_All_Gonna_Die', 'Play_It_As_It_Lays', 'The_Island_Between_Tides', 'The_Way_My_Way', 'You_Burn_Me', 'The_Corpse', 'Music_Night_Live_From_Nashville', 'Ufc_313_Pereira_Vs_Ankalaev']


2. Fetches detailed movie information (ratings, release date, box office, etc.) using the OMDb API:
o OMDb API: http://www.omdbapi.com/

In [57]:
import requests
from bs4 import BeautifulSoup
import csv
#from fpdf import FPDF it's not working in my machine 
import re
from datetime import datetime  # ✅ Added import for timestamp

API_KEY = "55989f0a"


# ---------------------- #
# 2️⃣ Extract Phase       #
# ---------------------- #
def extract_movies(titles):
    print("🚀 Fetching Data from OMDb API... 🎬")
    movies = []

    for title in titles:
        url = f"http://www.omdbapi.com/?t={title}&apikey={API_KEY}"
        try:
            response = requests.get(url, timeout=10)
            data = response.json()

            if data.get("Response") == "True":
                movies.append(data)
            else:
                print(f"❌ Movie not found: {title}")

        except requests.exceptions.RequestException as e:
            print(f"❌ Request Error for {title}: {e}")

    print(f"✅ Fetched {len(movies)} movies successfully!\n")
    return movies

# ---------------------- #
# 3️⃣ Transform Phase      #
# ---------------------- #
def transform_data(movies):
    print("🔄 Transforming Data... 🧹")
    transformed_movies = []  # ✅ Initialize the list

    for movie in movies:
         #TASK1 - Title Transformation ( •	Remove Special Characters: Use regular expressions to eliminate non-alphanumeric characters.)
           #( •	Standardized Case: Convert all titles to Title Case for consistency.)
        #(•	Trim Whitespaces: Remove leading/trailing spaces to maintain uniformity.)

        title_clean = re.sub(r'[^\w\s]', '', movie.get("Title", "").strip()).title() 
        
        #TASK2 - Release Date Transformation (•	Date Formatting: Replace spaces with hyphens to standardize the date format (e.g., DD MMM YYYY to DD-MMM-YYYY).
            #(•	Convert to Date Object: Use date parsing to convert text dates into proper date formats.)
            #(•	Handle Missing Dates: Replace missing dates with "Unknown.")

        release_date = movie.get("Released", "N/A")   
        formatted_date = (
            datetime.strptime(release_date, '%d %b %Y').strftime('%Y-%m-%d') 
            if release_date != "N/A" else "Unknown"
        )
         #TASK3 - Genre Transformation ( •Convert to Lowercase: Ensure all genres are in lowercase for consistency.)
                     #(•Split Genres: If multiple genres are present, separate them into a list.)
                     #(•Remove Duplicates: Ensure unique genre entries.)
        genres = list(set([genre.strip().lower() for genre in movie.get("Genre", "").split(",")])) 

        #TASK4- IMBD Rating Transformation (•Convert to Numeric: Change rating from text to a floating-point number.)
                    #(•	Round Off: Round ratings to one decimal place.)
                    #(•	Normalize: Optionally, normalize ratings on a scale of 0 to 1.)

        imdb_rating = round(float(movie.get("imdbRating", 0)), 1) if movie.get("imdbRating") != "N/A" else None  
        imdb_rating_normalized = imdb_rating / 10 if imdb_rating else None

         #TASK5- Actors Transformation   (•	Limit to Top 3: Display only the top three actors.)
                    #(•	Trim Spaces: Remove extra spaces around names.)
                    #(•	Sort Alphabetically (Optional): For consistency in display.)

        actors = ", ".join(sorted([actor.strip() for actor in movie.get("Actors", "").split(",")[:3]]))

         #TASK6 - BoxOffice Transofrmation   (•	Remove Currency Symbols: Eliminate $, ,, and other non-numeric characters.)
                    #(•	Convert to Numeric: Store as an integer for analysis.)
                    #(•	Handle Missing Data: Replace missing values with 0 or N/A)

        box_office_clean = re.sub(r'[^\d]', '', movie.get("BoxOffice", ""))  
        box_office_value = int(box_office_clean) if box_office_clean else 0

        #TASK7- Awards Transformation    (•	Extract Numbers: Identify and sum all numeric values related to awards won.)
                    #(•	Standardize Format: Display total awards won.)
                    #(•	Handle Missing Awards: Set to 0 if no data is available.)

        awards_text = movie.get("Awards", "")                                           # Getting awards text, default to empty if missing
        awards_numbers = [int(num) for num in re.findall(r'\d+', awards_text)]          # Extracting all numbers
        total_awards = sum(awards_numbers) if awards_numbers else 0                     # Sum all numbers, set to 0 if none found


        #TASK8- MetasScore Transformation  (•Convert to Integer: Change metascore to an integer for calculations.)
                    #(•	Normalize: Convert to a 0-1 scale by dividing by 100.)
                    #(•	Handle Missing Values: Replace "N/A" with None.)
        metascore = int(movie.get("Metascore", 0)) if movie.get("Metascore", "").isdigit() else None             # Converting to int if valid
        metascore_normalized = metascore / 100 if metascore is not None else None                                # Normalize (0-1 scale) or set to None


        #TASK9 - Language Transformation  (•Convert to Lowercase: Ensure all language names are in lowercase.)
                    #(•	Standardize Codes: Optionally convert to ISO language codes.)
                   #(•	Handle Missing Data: Replace missing languages with "Unknown.")

                
        language = movie.get("Language", "Unknown").lower()    # Gettinh the language field, default to "Unknown" if missing
        
        #Converting common language names to ISO codes (Example: English -> en, French -> fr)
        iso_language_mapping = {
            "english": "en",
            "french": "fr",
            "spanish": "es",
            "german": "de",
            "hindi": "hi",
            "chinese": "zh",
            "japanese": "ja",
            "korean": "ko"
        }
        
        # Convert known languages to ISO codes
        standardized_languages = [iso_language_mapping.get(lang.strip(), lang.strip()) 
                                  for lang in language.split(",")]
        
        # Rejoin languages into a single string
        language = ", ".join(standardized_languages)
                                              # Converting to lowercase, default to "Unknown" if missing


        #TASK10 - Production Transformation  (•	Remove Special Characters: Clean the production company names.)
                    #(•	Standardize Names: Correct common misspellings and standardize abbreviations.)
                    #(•	Handle Missing Data: Replace missing production companies with "Independent.")

        production = re.sub(r'[^\w\s]', '', movie.get("Production", "Independent")).strip()      # Removing special characters and trim spaces


        transformed_movies.append({
            "Title": title_clean,
            "Release Date": formatted_date,
            "Genres": genres,
            "IMDb Rating": imdb_rating,
            "Normalized IMDb Rating": imdb_rating_normalized,
            "Top 3 Actors": actors,
            "Box Office": box_office_value,
            "Total Awards": total_awards,
            "Metascore": metascore_normalized,
            "Language": language,
            "Production": production
        })

    print(f"✨ Transformation Complete for {len(transformed_movies)} movies!\n")
    return transformed_movies

# ---------------------- #
# 4️⃣ Load Phase (CSV)     #
# ---------------------- #
def load_to_csv(data):
    print("💾 Saving Data to CSV... 📊")
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"movies_{timestamp}.csv"  # ✅ Add timestamp to avoid overwrite

    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

    print(f"✅ Data successfully saved to {filename}!\n")


# ---------------------- #
# 🚀 Execute ETL Pipeline #
# ---------------------- #
if __name__ == "__main__":
    titles = fetch_movies_from_rotten_tomatoes()
    if titles:
        movies = extract_movies(titles)
        if movies:
            transformed_data = transform_data(movies)
            load_to_csv(transformed_data)
            
        else:
            print("⚠️ No movie data fetched from the API.")
    else:
        print("⚠️ No movie titles found during scraping.")


🍅 Fetching Latest Movies from Rotten Tomatoes...
✅ Fetched 11 movies from Rotten Tomatoes!

🚀 Fetching Data from OMDb API... 🎬
❌ Movie not found: The_Visitor_2024
❌ Movie not found: Were_All_Gonna_Die
❌ Movie not found: Music_Night_Live_From_Nashville
❌ Movie not found: Ufc_313_Pereira_Vs_Ankalaev
✅ Fetched 7 movies successfully!

🔄 Transforming Data... 🧹
✨ Transformation Complete for 7 movies!

💾 Saving Data to CSV... 📊
✅ Data successfully saved to movies_20250304_120059.csv!

